# 1.初始化PYU和SPU

In [3]:
import secretflow as sf
import spu
from secretflow.utils.testing import unused_tcp_port

sf.shutdown()
# Standalone Mode
sf.init(['alice', 'bob', 'carol', 'davy', 'eric'], address='local')
alice = sf.PYU('alice')
bob = sf.PYU('bob')
carol = sf.PYU('carol')

aby3_cluster_def = {
    'nodes': [
        {
            'party': 'alice',
            'address': f'127.0.0.1:{unused_tcp_port()}',
        },
        {
            'party': 'bob',
            # 'id': 'local:1', 
            'address': f'127.0.0.1:{unused_tcp_port()}'},
        {
            'party': 'carol',
            'address': f'127.0.0.1:{unused_tcp_port()}',
        },
    ],
    'runtime_config': {
        'protocol': spu.spu_pb2.ABY3,
        'field': spu.spu_pb2.FM64,
    },
}

my_spu = sf.SPU(aby3_cluster_def)

2024-05-19 09:05:06,205	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8266 


# 2.将数据输入PYU

In [5]:
x_plaintext = bank_account = [{'id': 12345, 'deposit': 1000.25}, {'id': 12345, 'deposit': 100000.25}]
x_alice_pyu = sf.to(alice, bank_account)
x_alice_pyu

# 3.PYUObject的操作

In [6]:
def add(x_in, y_in):
    return x_in + y_in


x_alice_pyu = alice(add)(x_alice_pyu, x_alice_pyu)

print(f"Plaintext Python Object: {x_plaintext}, PYU object: {x_alice_pyu}")
print(f"Reveal PYU object: {sf.reveal(x_alice_pyu)}")

Plaintext Python Object: [{'id': 12345, 'deposit': 1000.25}, {'id': 12345, 'deposit': 100000.25}], PYU object: <secretflow.device.device.pyu.PYUObject object at 0x7f03f8c857f0>
Reveal PYU object: [{'deposit': 1000.25, 'id': 12345}, {'deposit': 100000.25, 'id': 12345}, {'deposit': 1000.25, 'id': 12345}, {'deposit': 100000.25, 'id': 12345}]


# 4.将数据从PYU输入SPU

In [7]:
x_alice_spu = x_alice_pyu.to(my_spu)
print(f"SPU object: {x_alice_spu}")

print(f"Reveal SPU object: {sf.reveal(x_alice_spu)}")

SPU object: <secretflow.device.device.spu.SPUObject object at 0x7f041d23c4c0>
Reveal SPU object: [{'deposit': array(1000.25, dtype=float32), 'id': array(12345, dtype=int32)}, {'deposit': array(100000.25, dtype=float32), 'id': array(12345, dtype=int32)}, {'deposit': array(1000.25, dtype=float32), 'id': array(12345, dtype=int32)}, {'deposit': array(100000.25, dtype=float32), 'id': array(12345, dtype=int32)}]


# 5.SPUObject的操作

In [8]:
x_alice_spu = my_spu(add)(x_alice_spu, x_alice_spu)

print(f"SPU object: {x_alice_spu}")
print(f"Reveal SPU object: {sf.reveal(x_alice_spu)}")

SPU object: <secretflow.device.device.spu.SPUObject object at 0x7f03f8c496d0>
Reveal SPU object: [{'deposit': array(1000.25, dtype=float32), 'id': array(12345, dtype=int32)}, {'deposit': array(100000.25, dtype=float32), 'id': array(12345, dtype=int32)}, {'deposit': array(1000.25, dtype=float32), 'id': array(12345, dtype=int32)}, {'deposit': array(100000.25, dtype=float32), 'id': array(12345, dtype=int32)}, {'deposit': array(1000.25, dtype=float32), 'id': array(12345, dtype=int32)}, {'deposit': array(100000.25, dtype=float32), 'id': array(12345, dtype=int32)}, {'deposit': array(1000.25, dtype=float32), 'id': array(12345, dtype=int32)}, {'deposit': array(100000.25, dtype=float32), 'id': array(12345, dtype=int32)}]


 # 6.拉格朗日函数插值示例

In [9]:
def lagrange_interpolation(x_points, y_points, x):
    """
    使用拉格朗日插值多项式计算插值点的值。

    参数:
    x_points -- 已知数据点的x坐标列表
    y_points -- 已知数据点的y坐标列表
    x -- 需要插值的点的x坐标

    返回:
    插值点的y值
    """
    assert len(x_points) == len(y_points), "x_points和y_points的长度必须相同"

    def L(k, x):
        """计算第k个拉格朗日基函数的值"""
        numerator = 1
        denominator = 1
        for i in range(len(x_points)):
            if i != k:
                numerator *= (x - x_points[i])
                denominator *= (x_points[k] - x_points[i])
        return numerator / denominator

    y = 0
    for k in range(len(y_points)):
        y += y_points[k] * L(k, x)

    return y


# 示例使用
x_points = [0, 1, 2]
y_points = [1, 3, 2]
x = 1.5

result = lagrange_interpolation(x_points, y_points, x)
print(f"插值点 ({x}, {result})")


插值点 (1.5, 2.875)


In [12]:
x_points_spu = sf.to(alice, x_points).to(my_spu)
y_points_spu = sf.to(bob, y_points).to(my_spu)
x_spu = sf.to(carol, 1.5).to(my_spu)

y_spu = my_spu(lagrange_interpolation)(x_points_spu, y_points_spu, x_spu)

print(f"SPU object: {y_spu}")
print(f"Reveal SPU object: {sf.reveal(y_spu)}")

SPU object: <secretflow.device.device.spu.SPUObject object at 0x7f039c2e9880>
Reveal SPU object: 2.8749923706054688


# 7.关闭PYU和SPU

In [13]:
sf.shutdown()